In [170]:
from gurobipy import Model, GRB, tuplelist, quicksum
import json
import random

random.seed(3)

In [171]:
resources = ["t1"] # ,"t2", "t3"]
timeslices = [i for i in range(10)]
print(resources)
print(timeslices)

['t1']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [172]:
# Generate random availability for each resource
# For each resource, knock out 3 random hours
resource_slices = {}
for r in resources:
    empty_hours = random.sample(timeslices, 3)
    resource_slices[r] = [i for i in timeslices if i not in empty_hours]
print(resource_slices)

{'t1': [0, 1, 4, 5, 6, 7, 9]}


In [207]:
# Generate random requests
requests = {}
num_requests = 10

for i in range(num_requests):
    t1 = random.randint(0, 9)
    t2 = random.randint(0, 9)
    start = min([t1, t2])
    end = max([t1, t2])
    length = min(random.randint(1, 3), end-start)
    priority = random.randint(1, 5)
    requests[i] = {
            "window_start": start,
            "window_end": end,
            "length": length,
            "priority": priority
        }

for r in requests.items():
    print(r)

(0, {'window_start': 1, 'window_end': 5, 'length': 1, 'priority': 2})
(1, {'window_start': 3, 'window_end': 7, 'length': 2, 'priority': 4})
(2, {'window_start': 4, 'window_end': 6, 'length': 2, 'priority': 4})
(3, {'window_start': 5, 'window_end': 9, 'length': 3, 'priority': 5})
(4, {'window_start': 6, 'window_end': 9, 'length': 1, 'priority': 3})
(5, {'window_start': 0, 'window_end': 4, 'length': 3, 'priority': 2})
(6, {'window_start': 5, 'window_end': 8, 'length': 3, 'priority': 5})
(7, {'window_start': 1, 'window_end': 3, 'length': 2, 'priority': 5})
(8, {'window_start': 4, 'window_end': 4, 'length': 0, 'priority': 1})
(9, {'window_start': 7, 'window_end': 7, 'length': 0, 'priority': 3})


In [210]:
# Translate Requests into widx slices
slices = {}
for rid in range(len(requests)):

    r = requests[rid]
    w_start = r["window_start"]
    w_end = r["window_end"]
    length = r["length"]
    priority = r["priority"]

    print(rid, w_start, w_end, length)
    
    for resource, hours in resource_slices.items():
        # Determine if the current start can fit in the time slices for this resource
        initial_window = range(w_start, end+1)
        # print(initial_window)
        overlap = [t for t in hours if t in initial_window]
        # print(overlap)
        possible_windows = []
        for t1 in overlap:
            valid = True
            for section in range(length):
                if (t1 + section) not in overlap:
                    valid = False
                    break
            if valid:
                possible_windows.append(t1)
        print(possible_windows)
        for w in possible_windows:
            name = f"TEL-{resource}_ID-{rid}_ST-{w}"
            wnum = w + section

            slices[name] = {
                "rid": rid,
                "priority": priority,
                "resource": resource,
                "start": w
            }

    print()

for s in slices:
    print(s)
    print(slices[s])
    print()

0 1 5 1
[1, 4, 5, 6, 7]

1 3 7 2
[4, 5, 6]

2 4 6 2
[4, 5, 6]

3 5 9 3
[5]

4 6 9 1
[6, 7]

5 0 4 3
[4, 5]

6 5 8 3
[5]

7 1 3 2
[4, 5, 6]

8 4 4 0
[4, 5, 6, 7]

9 7 7 0
[7]

TEL-t1_ID-0_ST-1
{'rid': 0, 'priority': 2, 'resource': 't1', 'start': 1}

TEL-t1_ID-0_ST-4
{'rid': 0, 'priority': 2, 'resource': 't1', 'start': 4}

TEL-t1_ID-0_ST-5
{'rid': 0, 'priority': 2, 'resource': 't1', 'start': 5}

TEL-t1_ID-0_ST-6
{'rid': 0, 'priority': 2, 'resource': 't1', 'start': 6}

TEL-t1_ID-0_ST-7
{'rid': 0, 'priority': 2, 'resource': 't1', 'start': 7}

TEL-t1_ID-1_ST-4
{'rid': 1, 'priority': 4, 'resource': 't1', 'start': 4}

TEL-t1_ID-1_ST-5
{'rid': 1, 'priority': 4, 'resource': 't1', 'start': 5}

TEL-t1_ID-1_ST-6
{'rid': 1, 'priority': 4, 'resource': 't1', 'start': 6}

TEL-t1_ID-2_ST-4
{'rid': 2, 'priority': 4, 'resource': 't1', 'start': 4}

TEL-t1_ID-2_ST-5
{'rid': 2, 'priority': 4, 'resource': 't1', 'start': 5}

TEL-t1_ID-2_ST-6
{'rid': 2, 'priority': 4, 'resource': 't1', 'start': 6}

TEL-t1_ID-3

In [202]:
m = Model("Test Schedule")

In [203]:
requestLocations = tuplelist()
scheduled_vars = []
for r_name, r in slices.items():
    var = m.addVar(vtype=GRB.BINARY, name=str(r_name))
    scheduled_vars.append(var)
    requestLocations.append((r["rid"], r["wnum"], r["priority"], r["resource"], r["start"], var))

m.update()

In [204]:
# I DON'T KNOW WHY WE NEED THIS ONE
# for i, r in enumerate(slices):
    # scheduled_vars[i].start = r["wnum"]
# m.update()

In [205]:
# Constraint 1: One-of (only schedule one of these)
# Are we going to include these constraints in our final model? If there's no special cases, then they just turn into 
# Constraint 4 (which is why there is a skip_constraint2 flag).

In [206]:
# Constraint 2: And (only schedule if they can ALL be scheduled)
# I'm going to use this to ensure that all of an observation is scheduled
for rid in requests:
    for start in timeslices:
        for r in resources:
            match = requestLocations.select(rid, '*', '*', r, start, '*')
            if len(match) > 0:
                print(rid, r, start)
                print(match)
                andVar = m.addVar(vtype=GRB.BINARY, name=f"full_request_var_{rid}_{start}_{r}")
                m.update()
                nscheduled = quicksum(isScheduled for rid, wnum, priority, resource, start, isScheduled in match)
                m.addConstr(andVar == nscheduled, 

0 t1 5
<gurobi.tuplelist (2 tuples, 6 values each):
 ( 0 , 5 , 5 , t1 , 5 , <gurobi.Var t1_0_ST5_SEC0> )
 ( 0 , 6 , 5 , t1 , 5 , <gurobi.Var t1_0_ST5_SEC1> )
>
0 t1 6
<gurobi.tuplelist (2 tuples, 6 values each):
 ( 0 , 6 , 5 , t1 , 6 , <gurobi.Var t1_0_ST6_SEC0> )
 ( 0 , 7 , 5 , t1 , 6 , <gurobi.Var t1_0_ST6_SEC1> )
>
1 t1 1
<gurobi.tuplelist (1 tuples, 6 values each):
 ( 1 , 1 , 4 , t1 , 1 , <gurobi.Var t1_1_ST1_SEC0> )
>
1 t1 4
<gurobi.tuplelist (1 tuples, 6 values each):
 ( 1 , 4 , 4 , t1 , 4 , <gurobi.Var t1_1_ST4_SEC0> )
>
1 t1 5
<gurobi.tuplelist (1 tuples, 6 values each):
 ( 1 , 5 , 4 , t1 , 5 , <gurobi.Var t1_1_ST5_SEC0> )
>
1 t1 6
<gurobi.tuplelist (1 tuples, 6 values each):
 ( 1 , 6 , 4 , t1 , 6 , <gurobi.Var t1_1_ST6_SEC0> )
>
1 t1 7
<gurobi.tuplelist (1 tuples, 6 values each):
 ( 1 , 7 , 4 , t1 , 7 , <gurobi.Var t1_1_ST7_SEC0> )
>
1 t1 9
<gurobi.tuplelist (1 tuples, 6 values each):
 ( 1 , 9 , 4 , t1 , 9 , <gurobi.Var t1_1_ST9_SEC0> )
>
2 t1 4
<gurobi.tuplelist (1 tuples, 6 

In [196]:
# Constraint 4: Each request only scheduled once
for rid in requests:
    match = requestLocations.select(rid, '*', '*', '*', '*', '*')
    nscheduled = quicksum([isScheduled for reqid, wnum, priority, resource, start, isScheduled in match])
    m.addConstr(nscheduled <= 1, f'one_per_reqid_constraint_{rid}')
m.update()

In [197]:
# Constraint 3: Each timeslice should only have one request in it
for r in resource_slices:
    for wnum in resource_slices[r]:
        match = requestLocations.select('*', wnum, '*', r, '*', '*')
        if len(match) == 0:
            continue
        nscheduled = quicksum(isScheduled for i, w, p, r, start, isScheduled in match)
        m.addConstr(nscheduled <= 1, f'one_per_slice_constraint_{r}_{wnum}')
m.update()

In [198]:
objective = quicksum([isScheduled * (priority + 0.1/(wnum+1.0)) for req, wnum, priority, resource, start, isScheduled in requestLocations])

In [150]:
m.setObjective(objective)
m.modelSense = GRB.MAXIMIZE
m.params.MIPGap = 0.01
m.params.Method = 3
m.update()

Set parameter MIPGap to value 0.01
Set parameter Method to value 3


In [151]:
m.write("test_model.mps")

In [152]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 25 columns and 50 nonzeros
Model fingerprint: 0xc21d554a
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 21.2134524
Presolve removed 17 rows and 25 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 22.2235 21.2135 

Optimal solution found (tolerance 1.00e-02)
Best objective 2.222345238095e+01, best bound 2.222345238095e+01, gap 0.0000%


In [153]:
rxf = [isScheduled.x for rid, wnum, priority, resource, start, isScheduled in requestLocations]

In [154]:
print("ID, w, p, reso, s")
for i in range(len(requestLocations)):
    if rxf[i] == 1:
        print(requestLocations[i])

ID, w, p, reso, s
(0, 5, 5, 't1', 5, <gurobi.Var t1_0_ST5_SEC0 (value 1.0)>)
(1, 9, 4, 't1', 1, <gurobi.Var t1_1_ST9_SEC0 (value 1.0)>)
(2, 4, 2, 't1', 4, <gurobi.Var t1_2_ST4_SEC0 (value 1.0)>)
(3, 7, 4, 't1', 7, <gurobi.Var t1_3_ST7_SEC0 (value 1.0)>)
(6, 1, 1, 't1', 1, <gurobi.Var t1_6_ST1_SEC0 (value 1.0)>)
(7, 0, 4, 't1', 0, <gurobi.Var t1_7_ST0_SEC0 (value 1.0)>)
(9, 6, 2, 't1', 6, <gurobi.Var t1_9_ST6_SEC0 (value 1.0)>)


In [155]:
print(resource_slices)

{'t1': [0, 1, 4, 5, 6, 7, 9]}


In [156]:
requests

{0: {'start': 5, 'end': 9, 'length': 2, 'priority': 5},
 1: {'start': 1, 'end': 9, 'length': 1, 'priority': 4},
 2: {'start': 4, 'end': 8, 'length': 1, 'priority': 2},
 3: {'start': 7, 'end': 8, 'length': 1, 'priority': 4},
 4: {'start': 2, 'end': 6, 'length': 1, 'priority': 2},
 5: {'start': 6, 'end': 8, 'length': 2, 'priority': 1},
 6: {'start': 1, 'end': 2, 'length': 1, 'priority': 1},
 7: {'start': 0, 'end': 4, 'length': 2, 'priority': 4},
 8: {'start': 6, 'end': 9, 'length': 3, 'priority': 4},
 9: {'start': 6, 'end': 9, 'length': 2, 'priority': 2}}